In [1]:
import pickle
import itertools
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [342]:
with open("curves_by/one_leaf.curves", "rb") as pickle_file:
    curve_fragment = pickle.load(pickle_file)

In [343]:
cfs = [[],[],[],[],[],[],[],[]]
for cols in curve_fragment:
    for i in range(len(cols)):
        if i == 0:
            cfs[0] += cols[i]
        if i == 1:
            cfs[1] += cols[i]
        if i == 2:
            cfs[2] += cols[i]
        if i == 3:
            cfs[3] += cols[i]
        if i == 4:
            cfs[4] += cols[i]
        if i == 5:
            cfs[5] += cols[i]
        if i == 6:
            cfs[6] += cols[i]
        if i == 7:
            cfs[7] += cols[i]

In [354]:
from scipy import interpolate
color_arr = np.array([[0,255,0],[0,255,0],[0,0,255],
                             [255,255,0],[255,0,255],[0,255,255],
                             [127,127,127],[127,0,127],[0,127,127]],dtype = np.int16)

_cfs = []

for i in range(len(cfs)):
    color_cfs = []
    for j, frag in enumerate(cfs[i]):
        if frag.size == 0:
            continue
        #print(frag)
        x = frag[:,0]
        y = frag[:,1]
        z = frag[:,2]
        data = [x,y,z]
        try:
            tck, u= interpolate.splprep(data, k=3)
        except ValueError:
            pass
        except TypeError:
            pass
        new = interpolate.splev(u, tck, der=0)
        c = color_arr[i]/255
        if j % 20 == 0:
            color_cfs.append(np.array([new[0], new[2], new[1]]).transpose(1,0))
    _cfs.append(color_cfs)

In [355]:
mesh = o3d.io.read_triangle_mesh("polygon/plant3.ply")
edges = mesh.get_non_manifold_edges(allow_boundary_edges=False)

In [356]:
def edges_to_lineset(mesh, edges, color):
    ls = o3d.geometry.LineSet()
    ls.points = mesh.vertices
    ls.lines = edges
    colors = np.empty((np.asarray(edges).shape[0], 3))
    colors[:] = color
    ls.colors = o3d.utility.Vector3dVector(colors)
    return ls

In [357]:
def rotation_mat(angle):
    Rx = np.array([[1,0,0],
                 [0, np.cos(angle[0]), -np.sin(angle[0])],
                 [0, np.sin(angle[0]), np.cos(angle[0])]])

    Ry = np.array([[np.cos(angle[1]), 0, np.sin(angle[1])],
                 [0,1,0],
                 [-np.sin(angle[1]), 0, np.cos(angle[1])]])

    Rz = np.array([[np.cos(angle[2]), -np.sin(angle[2]), 0],
                 [np.sin(angle[2]), np.cos(angle[2]), 0],
                 [0,0,1]])
    return Rz@Rx@Ry


In [358]:
geoms = [mesh]
geoms.append(edges_to_lineset(mesh, edges, (1, 0, 0)))

In [359]:
#o3d.visualization.draw_geometries(geoms)

In [360]:
angle = [np.pi, np.pi, 0] # pcd.rotate()
#angle = [-np.pi/2, np.pi/2, np.pi/2]
R = rotation_mat(angle)
R_mirror = np.array([[-1,0,0],
                    [0,1,0],
                    [0,0,1]])

In [361]:
len(_cfs)

8

In [362]:
geo = []
for j, color in enumerate(_cfs):
    for l_c in color:
        points = l_c
        lines = [[i,i+1] for i in range(len(points)-1)]
        colors = [color_arr[j]/255 for i in range(len(lines))]
        #print(len(points),len(lines),len(colors))

        line_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(points),
            lines=o3d.utility.Vector2iVector(lines),
        )
        line_set.colors = o3d.utility.Vector3dVector(colors)
        line_set.rotate(np.linalg.inv(R),(0,0,0))
        line_set.scale(10,(0,0,0))
        geo.append(line_set)

o3d.visualization.draw_geometries(geo)

In [341]:
geo.append(mesh)
o3d.visualization.draw_geometries(geo)

In [195]:
flatten_curve_frag = np.array(list(itertools.chain.from_iterable(_cfs)))
#flatten_curve_frag = (np.linalg.inv(R_mirror)@np.linalg.inv(R)@flatten_curve_frag.T).T

point_num = flatten_curve_frag.shape[0]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(flatten_curve_frag)
pcd.colors = o3d.utility.Vector3dVector(np.repeat(np.array((0, 1, 0)).reshape(1,-1), point_num, axis=0))
#pcd.rotate(R_mirror,(0,0,0))
pcd.rotate(np.linalg.inv(R),(0,0,0))
pcd.scale(10,(0,0,0))
geoms.append(pcd)

In [196]:
o3d.visualization.draw_geometries(geoms)

In [182]:
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [167]:
np.asarray(pcd.points)

array([[2.48615587, 2.2975443 , 1.07655068],
       [2.49763071, 2.28802732, 1.09351589],
       [2.50086806, 2.28528549, 1.09794999],
       ...,
       [2.14656379, 2.26655423, 1.34958684],
       [2.12544096, 2.26875637, 1.36164059],
       [2.10515791, 2.27130045, 1.37321302]])